In [ ]:
import trimesh
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import pandas as pd
from scipy.spatial import KDTree
from scipy.spatial.transform import Rotation


In [ ]:
BINS = 1024
OUTPUT_ROOT = "Outputs"
MESH_DIR = "meshes"
OUTPUT_ROOT_TASK4 = "Outputs_Task4"
NUM_TRANSFORMATIONS = 5
K_NEIGHBORS = 10
ADAPTIVE_DENSITY_FACTOR = 1.0

In [ ]:
def load_and_analyze(filepath):
    #Initial print
    print(f"\n{'='*30}")
    print(f"Analyzing: {os.path.basename(filepath)}")
    print('='*30)

    # Mesh loading
    mesh = trimesh.load(filepath, process=False)
    vertices = np.array(mesh.vertices)  
    faces = np.array(mesh.faces)
        
    num_vertices = vertices.shape[0]
    num_faces = faces.shape[0]
    
    print(f"Number of vertices: {num_vertices}")
    print(f"Number of faces: {num_faces}")
    
    #per-axis statistics
    axis_names = np.array(['X', 'Y', 'Z'])
    mins = np.min(vertices, axis=0)
    maxs = np.max(vertices, axis=0)
    means = np.mean(vertices, axis=0)
    stds = np.std(vertices, axis=0)
        
    for i, axis in enumerate(axis_names):
        print(f"\n{axis}-axis:")
        print(f"  Min: {mins[i]:.4f}")
        print(f"  Max: {maxs[i]:.4f}")
        print(f"  Mean: {means[i]:.4f}")
        print(f"  Std Dev: {stds[i]:.4f}")
    
    # Additional analysis
    print(f"\nBounding box dimensions:")
    bbox_dims = maxs - mins
    print(f"Width (X): {bbox_dims[0]:.4f}")
    print(f"Depth (Y): {bbox_dims[1]:.4f}")
    print(f"Height (Z): {bbox_dims[2]:.4f}")
    
    print(f"\nMesh centroid: ({means[0]:.4f}, {means[1]:.4f}, {means[2]:.4f})")
    
    # Visualization                                
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    ax.plot_trisurf(vertices[:, 0], vertices[:, 1], vertices[:, 2], triangles=faces, cmap='viridis', alpha=0.8, edgecolor='none')
    
    ax.set_title(f'{os.path.basename(filepath)}')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    
    plt.tight_layout()
    plt.show()
    
    return mesh, vertices

In [ ]:
meshes = "meshes/"
mesh_samples = np.array([i for i in os.listdir(meshes) if i.endswith('.obj')])

print(f"Found {len(mesh_samples)} sample mesh files")

In [ ]:
for sample_filename in mesh_samples:
    filepath = os.path.join(meshes, sample_filename)
    mesh, vertices = load_and_analyze(filepath)

In [ ]:
def minmax_normalize(vertices):
    """
    Min–Max Normalization: Scale vertex coordinates into [0, 1] per axis.
    """
    v_min = vertices.min(axis=0)
    v_max = vertices.max(axis=0)
    # prevent divide-by-zero
    range_ = v_max - v_min
    if np.any(range_ == 0):
        print("⚠️ Warning: Zero range axis detected; skipping normalization for that axis.")
        range_[range_ == 0] = 1.0
    normalized = (vertices - v_min) / range_
    return normalized, v_min, v_max

def unit_sphere_normalize(vertices):
    """
    Unit Sphere Normalization: Center vertices at origin and scale so that 
    the farthest vertex lies on a unit sphere (‖v‖ ≤ 1).
    """
    centroid = vertices.mean(axis=0)
    centered = vertices - centroid
    scale = np.max(np.linalg.norm(centered, axis=1))
    if scale == 0 or np.isclose(scale, 0): 
        scale = 1.0
    normalized = centered / scale
    
    # Verify normalization
    max_norm = np.max(np.linalg.norm(normalized, axis=1))
    assert np.isclose(max_norm, 1.0, atol=1e-6), f"Normalization failed: max_norm={max_norm}"
    
    return normalized, centroid, scale

In [ ]:
def quantize(normalized, bins=1024):
    """Discretize normalized coordinates into bins."""
    norm_copy = normalized.copy()
    
    # Determine if we need to shift based on range
    data_min = norm_copy.min()
    data_max = norm_copy.max()
    
    # If data spans negative values, it's in [-1, 1] range
    shifted = (data_min < -0.01)  # Use small threshold for numerical stability
    
    if shifted:
        # Shift from [-1, 1] to [0, 1]
        norm_copy = (norm_copy + 1.0) / 2.0
    
    # Ensure we're in [0, 1]
    norm_copy = np.clip(norm_copy, 0.0, 1.0)
    
    # Quantize to integer bins
    quantized = np.floor(norm_copy * (bins - 1)).astype(np.int64)
    quantized = np.clip(quantized, 0, bins - 1)  # Extra safety
    
    return quantized, shifted


In [ ]:
def visualize_task2(norm_mm, norm_us, quant_mm, quant_us, faces, sample_name, sample_dir):
    """
    Generateing a 2×2 subplot for each mesh:
    Row 1: Min–Max Normalized | Unit Sphere Normalized
    Row 2: Min–Max Quantized  | Unit Sphere Quantized
    """
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
    import numpy as np, os

    # #Downsample for speed
    # def maybe_downsample(verts, max_verts=8000):
    #     if len(verts) > max_verts:
    #         idx = np.random.choice(len(verts), max_verts, replace=False)
    #         return verts[idx]
    #     return verts
    """ Removed the downsampling function and passing the values as it is."""
    norm_mm_vis = norm_mm
    norm_us_vis = norm_us
    quant_mm_array, _ = quant_mm  # Extracting the array part
    quant_us_array, _ = quant_us  # Extracting the array part

    norm_mm_vis = norm_mm
    norm_us_vis = norm_us
    quant_mm_vis = quant_mm_array.astype(float)  # FIXED
    quant_us_vis = quant_us_array.astype(float)

    # --- Create figure ---
    fig, axs = plt.subplots(2, 2, figsize=(14, 12), subplot_kw={'projection': '3d'})
    fig.suptitle(f'Task 2 Results: {sample_name}', fontsize=15, fontweight='bold')

    
    def set_equal_axes(ax, data):
        mins, maxs = data.min(axis=0), data.max(axis=0)
        rng = maxs - mins
        mid = (maxs + mins) / 2
        max_range = rng.max() / 2
        ax.set_xlim(mid[0]-max_range, mid[0]+max_range)
        ax.set_ylim(mid[1]-max_range, mid[1]+max_range)
        ax.set_zlim(mid[2]-max_range, mid[2]+max_range)

  
    plots = [
        (axs[0,0], norm_mm_vis, 'Min–Max Normalized', 'skyblue'),
        (axs[0,1], norm_us_vis, 'Unit Sphere Normalized', 'lightgreen'),
        (axs[1,0], quant_mm_vis, 'Min–Max Quantized', 'orange'),
        (axs[1,1], quant_us_vis, 'Unit Sphere Quantized', 'pink')
    ]

    for ax, data, title, color in plots:
        ax.scatter(data[:,0], data[:,1], data[:,2], s=5, c=color, alpha=0.7)
        ax.set_title(title, fontweight='bold')
        ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')
        set_equal_axes(ax, data)
        ax.grid(False)

    plt.tight_layout(rect=[0, 0, 1, 0.96])

 
    plot_path = os.path.join(sample_dir, "plots", "task2_results.png")
    os.makedirs(os.path.dirname(plot_path), exist_ok=True)
    plt.savefig(plot_path, dpi=200, bbox_inches='tight')
    plt.close(fig)

    print(f"Visualization saved {plot_path}")


In [ ]:
def process_mesh_task2(mesh_path, sample_name):
    """
    Process a single mesh for Task 2:
    1. Load mesh
    2. Apply both normalization methods
    3. Quantize both
    4. Save all four meshes (.ply)
    5. Generate visualization
    """
    print(f"\n{'='*60}")
    print(f"Processing: {sample_name}")
    print(f"{'='*60}")
    
    # Load mesh
    mesh = trimesh.load(mesh_path, process=False)
    vertices = np.array(mesh.vertices)
    faces = np.array(mesh.faces)
    
    print(f"Vertices: {len(vertices)}, Faces: {len(faces)}")
    
    # Apply both normalization methods
    print("\n[1] Normalizing...")
    norm_mm, vmin, vmax = minmax_normalize(vertices)
    norm_us, centroid, scale = unit_sphere_normalize(vertices)
    print(f" Min–Max range: [{norm_mm.min():.3f}, {norm_mm.max():.3f}]")
    print(f" Unit Sphere range: [{norm_us.min():.3f}, {norm_us.max():.3f}]")

    # Making sure to save all these vmin,vmax,norm_us, etc in a json file for future use in task 3.
    sample_dir = os.path.join(OUTPUT_ROOT, sample_name)
    os.makedirs(sample_dir, exist_ok=True)

    # Quantize both
    print(f"\n[2] Quantizing with {BINS} bins...")
    quant_mm, shifted_mm = quantize(norm_mm, bins=BINS)
    quant_us, shifted_us = quantize(norm_us, bins=BINS)

    params = {
        "vmin": vmin.tolist(),
        "vmax": vmax.tolist(),
        "centroid": centroid.tolist(),
        "scale": float(scale),
        "bins": BINS,
        "shifted_mm": shifted_mm,   # Min–Max normalization already in [0,1]
        "shifted_us": shifted_us,    # Unit Sphere normalization mapped from [-1,1] → [0,1]
        }             

    params_path = os.path.join(sample_dir, "normalization_params.json")
    
    with open(params_path, "w") as f:
        json.dump(params, f, indent=4)
        print(f" Saved normalization parameters: {params_path}")
    
    

    print(f" Min–Max quantized: [{quant_mm.min()}, {quant_mm.max()}]")
    print(f" Unit Sphere quantized: [{quant_us.min()}, {quant_us.max()}]")
    
    # Create output directories
    sample_dir = os.path.join(OUTPUT_ROOT, sample_name)
    os.makedirs(os.path.join(sample_dir, "normalized"), exist_ok=True)
    os.makedirs(os.path.join(sample_dir, "quantized"), exist_ok=True)
    os.makedirs(os.path.join(sample_dir, "plots"), exist_ok=True)
    
    # Save all four meshes
    print("\n[3] Saving meshes...")
    trimesh.Trimesh(norm_mm, faces, process=False).export(
        os.path.join(sample_dir, "normalized", "minmax.ply"))
    trimesh.Trimesh(norm_us, faces, process=False).export(
        os.path.join(sample_dir, "normalized", "unitsphere.ply"))
    trimesh.Trimesh(quant_mm.astype(float), faces, process=False).export(
        os.path.join(sample_dir, "quantized", "minmax.ply"))
    trimesh.Trimesh(quant_us.astype(float), faces, process=False).export(
        os.path.join(sample_dir, "quantized", "unitsphere.ply"))
    
    print(f"  Saved normalized meshes: {sample_dir}/normalized/")
    print(f"  Saved quantized meshes: {sample_dir}/quantized/")

    # Generate visualization
    print("\n[4] Creating visualization...")
    visualize_task2(norm_mm, norm_us, quant_mm, quant_us, faces, sample_name, sample_dir)
    
    print(f"\n{'='*60}")
    print(f"Completed: {sample_name}")
    print(f"{'='*60}\n")


In [ ]:
def process_all_meshes():
    """
    Loop through all meshes in the dataset and process each for Task 2.
    """
    # Find all .obj files in MESH_DIR
    mesh_files = [f for f in os.listdir(MESH_DIR) if f.endswith('.obj')]
    
    if not mesh_files:
        print(f"No .obj files found in {MESH_DIR}/")
        return
    
    print(f"\n{'='*70}")
    print(f"TASK 2: NORMALIZE & QUANTIZE MESHES")
    print(f"{'='*70}")
    print(f"Found {len(mesh_files)} mesh file(s)")
    print(f"Output directory: {OUTPUT_ROOT}/")
    print(f"Bins: {BINS}")
    
    # Process each mesh
    for mesh_file in mesh_files:
        mesh_path = os.path.join(MESH_DIR, mesh_file)
        sample_name = os.path.splitext(mesh_file)[0]

        
        try:
            process_mesh_task2(mesh_path, sample_name)  
        except Exception as e:
            print(f"\n✗ ERROR processing {mesh_file}: {e}")
            import traceback
            traceback.print_exc()
    
    print(f"\n{'='*70}")
    print(f"TASK 2 COMPLETE")
    print(f"{'='*70}")
    print(f"\n Deliverables for each mesh:")
    print(f"   normalized/minmax.ply")
    print(f"   normalized/unitsphere.ply")
    print(f"   quantized/minmax.ply")
    print(f"   quantized/unitsphere.ply")
    print(f"   plots/task2_results.png")
    print(f"\n{'='*70}\n")

In [ ]:
if __name__ == "__main__":
    process_all_meshes()

Normalization is done to put the mesh in a common scale so that it can be better processed. The 2 such way to perform normalization on a mesh are as follows:-

1. Min-max Normalization

In Min-max we take the minimum and the maximum points in each axis x,y,z and create a box of 0 to 1 to squash these mesh coordinates in the box. Using min-max usually distorts the mesh structure as it needs to put all the coordinates into this box. Hence mesh structure is NOT preserved.

2. Unit Sphere Normalization

In Unit-Sphere we take a center point in the mesh and create a sphere. We then try to shrink the mesh structer uniformaly such that the farthest vertex is on the surface of the sphere radius=1. The structure is preserved and normalization is also achieved. Hence mesh structure is preserved and Normalization is achieved.

-> Unit Sphere Normalization preserves the Mesh Structure and Normalize the mesh at the same time.




In [ ]:
def denormalize_minmax(dequantized, vmin, vmax):
    """Denormalize Min-Max back to original scale."""
    return dequantized * (vmax - vmin) + vmin

def denormalize_unitsphere(dequantized, centroid, scale):
    """Denormalize Unit Sphere back to original scale."""
    return dequantized * scale + centroid

def dequantize(quantized, bins=1024, shifted=False):
    """Dequantize back to normalized coordinates."""
    # Convert to float and normalize to [0, 1]
    dequantized = quantized.astype(float) / (bins - 1)
    dequantized = np.clip(dequantized, 0.0, 1.0)  # Safety clamp
    
    if shifted:
        # Shift back from [0, 1] to [-1, 1]
        dequantized = dequantized * 2.0 - 1.0
    
    return dequantized

def compute_errors(original, reconstructed):
    """Compute reconstruction error metrics."""
    mse = np.mean((original - reconstructed) ** 2)
    mae = np.mean(np.abs(original - reconstructed))
    mse_axis = np.mean((original - reconstructed) ** 2, axis=0)
    mae_axis = np.mean(np.abs(original - reconstructed), axis=0)
    
    return {
        "mse": mse,
        "mae": mae,
        "mse_axis": mse_axis,
        "mae_axis": mae_axis
    }


In [ ]:
def visualize_reconstruction(original, recon_mm, recon_us, sample_name, sample_dir):
    """Visualize original vs reconstructed meshes (1×3 layout)."""
    from mpl_toolkits.mplot3d import Axes3D
    
    fig, axs = plt.subplots(1, 3, figsize=(18, 6), subplot_kw={'projection': '3d'})
    fig.suptitle(f'Task 3: Reconstruction - {sample_name}', fontsize=15, fontweight='bold')
    
    def set_equal_axes(ax, data):
        mins, maxs = data.min(axis=0), data.max(axis=0)
        rng = maxs - mins
        mid = (maxs + mins) / 2
        max_range = rng.max() / 2
        ax.set_xlim(mid[0]-max_range, mid[0]+max_range)
        ax.set_ylim(mid[1]-max_range, mid[1]+max_range)
        ax.set_zlim(mid[2]-max_range, mid[2]+max_range)
    
    plots = [
        (axs[0], original, 'Original', 'gray'),
        (axs[1], recon_mm, 'Min–Max Reconstructed', 'green'),
        (axs[2], recon_us, 'Unit Sphere Reconstructed', 'blue')
    ]
    
    for ax, data, title, color in plots:
        ax.scatter(data[:,0], data[:,1], data[:,2], s=5, c=color, alpha=0.7)
        ax.set_title(title, fontweight='bold')
        ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')
        set_equal_axes(ax, data)
        ax.grid(False)
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plot_path = os.path.join(sample_dir, "plots", "task3_reconstruction.png")
    plt.savefig(plot_path, dpi=200, bbox_inches='tight')
    plt.close(fig)
    print(f"  Reconstruction visualization saved: {plot_path}")

def plot_error_per_axis(errors_mm, errors_us, sample_name, sample_dir):
    """Plot reconstruction error per axis (bar chart)."""
    axes_labels = ['X', 'Y', 'Z']
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    fig.suptitle(f'Task 3: Reconstruction Error per Axis - {sample_name}', 
                 fontsize=14, fontweight='bold')
    
    # MSE per axis
    x = np.arange(len(axes_labels))
    width = 0.35
    
    ax1.bar(x - width/2, errors_mm['mse_axis'], width, label='Min–Max', 
            alpha=0.8, color='green')
    ax1.bar(x + width/2, errors_us['mse_axis'], width, label='Unit Sphere', 
            alpha=0.8, color='blue')
    ax1.set_xlabel('Axis')
    ax1.set_ylabel('MSE')
    ax1.set_title('Mean Squared Error per Axis')
    ax1.set_xticks(x)
    ax1.set_xticklabels(axes_labels)
    ax1.legend()
    ax1.grid(axis='y', alpha=0.3)
    
    # MAE per axis
    ax2.bar(x - width/2, errors_mm['mae_axis'], width, label='Min–Max', 
            alpha=0.8, color='green')
    ax2.bar(x + width/2, errors_us['mae_axis'], width, label='Unit Sphere', 
            alpha=0.8, color='blue')
    ax2.set_xlabel('Axis')
    ax2.set_ylabel('MAE')
    ax2.set_title('Mean Absolute Error per Axis')
    ax2.set_xticks(x)
    ax2.set_xticklabels(axes_labels)
    ax2.legend()
    ax2.grid(axis='y', alpha=0.3)
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plot_path = os.path.join(sample_dir, "plots", "task3_error_plot.png")
    plt.savefig(plot_path, dpi=200, bbox_inches='tight')
    plt.close(fig)
    print(f"  Error plot saved: {plot_path}")


In [ ]:
def process_mesh_task3(mesh_path, sample_name):
    """Process a single mesh for Task 3: Dequantize, Denormalize, Measure Error."""
    print(f"\n{'='*60}")
    print(f"Task 3 Processing: {sample_name}")
    print(f"{'='*60}")
    
    sample_dir = os.path.join(OUTPUT_ROOT, sample_name)
    
    # STEP 2 — Load required data
    print("\n[1] Loading data...")
    
    # Load original mesh
    mesh = trimesh.load(mesh_path, process=False)
    original_vertices = np.array(mesh.vertices)
    print(f"  Loaded original: {len(original_vertices)} vertices")
    
    # Load normalization parameters
    params_path = os.path.join(sample_dir, "normalization_params.json")
    if not os.path.exists(params_path):
        print(f"  Parameters not found: {params_path}")
        return None
    
    with open(params_path, 'r') as f:
        params = json.load(f)
    
    vmin = np.array(params['vmin'])
    vmax = np.array(params['vmax'])
    centroid = np.array(params['centroid'])
    scale = params['scale']
    bins = params['bins']
    shifted_mm = params['shifted_mm']
    shifted_us = params['shifted_us']
    print(f"  Loaded normalization parameters")
    
    # Load quantized meshes
    quant_mm_mesh = trimesh.load(
        os.path.join(sample_dir, "quantized", "minmax.ply"), process=False)
    quant_us_mesh = trimesh.load(
        os.path.join(sample_dir, "quantized", "unitsphere.ply"), process=False)
    
    quant_mm = np.array(quant_mm_mesh.vertices)
    quant_us = np.array(quant_us_mesh.vertices)
    print(f"  Loaded quantized meshes")
    
    # STEP 3 — Dequantize
    print("\n[2] Dequantizing...")
    dequant_mm = dequantize(quant_mm.astype(int), bins=bins, shifted=shifted_mm)
    dequant_us = dequantize(quant_us.astype(int), bins=bins, shifted=shifted_us)
    print(f"  Dequantized both meshes")
    
    # STEP 4 — Denormalize to original scale
    print("\n[3] Denormalizing to original scale...")
    recon_mm = denormalize_minmax(dequant_mm, vmin, vmax)
    recon_us = denormalize_unitsphere(dequant_us, centroid, scale)
    print(f"  Reconstructed Min–Max mesh")
    print(f"  Reconstructed Unit Sphere mesh")
    
    # STEP 5 — Compute reconstruction errors
    print("\n[4] Computing reconstruction errors...")
    errors_mm = compute_errors(original_vertices, recon_mm)
    errors_us = compute_errors(original_vertices, recon_us)
    
    print(f"\n  Min–Max Errors:")
    print(f"    MSE: {errors_mm['mse']:.8f}")
    print(f"    MAE: {errors_mm['mae']:.8f}")
    
    print(f"\n  Unit Sphere Errors:")
    print(f"    MSE: {errors_us['mse']:.8f}")
    print(f"    MAE: {errors_us['mae']:.8f}")
    
    # STEP 6 — Visualize reconstructed meshes
    print("\n[5] Creating reconstruction visualization...")
    visualize_reconstruction(original_vertices, recon_mm, recon_us, 
                           sample_name, sample_dir)
    
    # STEP 7 — Plot error per axis
    print("\n[6] Creating error plots...")
    plot_error_per_axis(errors_mm, errors_us, sample_name, sample_dir)
    
    print(f"\n{'='*60}")
    print(f"Task 3 Completed: {sample_name}")
    print(f"{'='*60}\n")
    
    return {
        "mesh": sample_name,
        "mm_mse": errors_mm['mse'],
        "mm_mae": errors_mm['mae'],
        "us_mse": errors_us['mse'],
        "us_mae": errors_us['mae'],
        "mm_mse_x": errors_mm['mse_axis'][0],
        "mm_mse_y": errors_mm['mse_axis'][1],
        "mm_mse_z": errors_mm['mse_axis'][2],
        "us_mse_x": errors_us['mse_axis'][0],
        "us_mse_y": errors_us['mse_axis'][1],
        "us_mse_z": errors_us['mse_axis'][2]
    }

In [ ]:
def process_all_meshes_task3():
    """Process all meshes for Task 3."""
    mesh_files = [f for f in os.listdir(MESH_DIR) if f.endswith('.obj')]
    
    if not mesh_files:
        print(f"No .obj files found in {MESH_DIR}/")
        return
    
    print(f"\n{'='*70}")
    print(f"TASK 3: DEQUANTIZE, DENORMALIZE, AND MEASURE ERROR")
    print(f"{'='*70}")
    print(f"Found {len(mesh_files)} mesh file(s)")
    
    all_results = []
    
    for mesh_file in mesh_files:
        mesh_path = os.path.join(MESH_DIR, mesh_file)
        sample_name = os.path.splitext(mesh_file)[0]
        
        try:
            result = process_mesh_task3(mesh_path, sample_name)
            if result:
                all_results.append(result)
        except Exception as e:
            print(f"\nERROR processing {mesh_file}: {e}")
            import traceback
            traceback.print_exc()
    
    # STEP 8 — Aggregate results
    if all_results:
        print(f"\n{'='*70}")
        print(f"AGGREGATE RESULTS - ALL MESHES")
        print(f"{'='*70}")
        
        # Summary table
        print(f"\n┌{'─'*30}┬{'─'*13}┬{'─'*13}┐")
        print(f"│ {'Normalization Type':<28} │ {'Mean MSE':>11} │ {'Mean MAE':>11} │")
        print(f"├{'─'*30}┼{'─'*13}┼{'─'*13}┤")
        
        avg_mm_mse = np.mean([r['mm_mse'] for r in all_results])
        avg_mm_mae = np.mean([r['mm_mae'] for r in all_results])
        avg_us_mse = np.mean([r['us_mse'] for r in all_results])
        avg_us_mae = np.mean([r['us_mae'] for r in all_results])
        
        print(f"│ {'Min–Max':<28} │ {avg_mm_mse:>11.6f} │ {avg_mm_mae:>11.6f} │")
        print(f"│ {'Unit Sphere':<28} │ {avg_us_mse:>11.6f} │ {avg_us_mae:>11.6f} │")
        print(f"└{'─'*30}┴{'─'*13}┴{'─'*13}┘\n")
        
        # Save CSV
        try:
            import pandas as pd
            df = pd.DataFrame(all_results)
            csv_path = os.path.join(OUTPUT_ROOT, "summary_task3_errors.csv")
            df.to_csv(csv_path, index=False)
            print(f" Summary saved to: {csv_path}")
        except ImportError:
            # Fallback without pandas
            csv_path = os.path.join(OUTPUT_ROOT, "summary_task3_errors.csv")
            with open(csv_path, 'w') as f:
                f.write("mesh,mm_mse,mm_mae,us_mse,us_mae,mm_mse_x,mm_mse_y,mm_mse_z,us_mse_x,us_mse_y,us_mse_z\n")
                for r in all_results:
                    f.write(f"{r['mesh']},{r['mm_mse']:.8f},{r['mm_mae']:.8f},"
                           f"{r['us_mse']:.8f},{r['us_mae']:.8f},"
                           f"{r['mm_mse_x']:.8f},{r['mm_mse_y']:.8f},{r['mm_mse_z']:.8f},"
                           f"{r['us_mse_x']:.8f},{r['us_mse_y']:.8f},{r['us_mse_z']:.8f}\n")
            print(f" Summary saved to: {csv_path}")
        
        # STEP 9 — Generate conclusion
        conclusion = f"""
{'='*70}
TASK 3: RECONSTRUCTION ERROR ANALYSIS - CONCLUSION
{'='*70}

Summary Statistics:
  Min-Max Normalization:
    Mean MSE: {avg_mm_mse:.8f}
    Mean MAE: {avg_mm_mae:.8f}

  Unit Sphere Normalization:
    Mean MSE: {avg_us_mse:.8f}
    Mean MAE: {avg_us_mae:.8f}

Analysis:
"""
        
        if avg_mm_mse < avg_us_mse:
            diff_pct = ((avg_us_mse - avg_mm_mse) / avg_us_mse) * 100
            conclusion += f"""
   Min-Max normalization consistently resulted in lower reconstruction 
   errors (MSE ≈ {avg_mm_mse:.8f}) compared to Unit Sphere normalization.
    
   Min-Max achieves {diff_pct:.1f}% lower MSE on average
   This suggests axis-independent scaling better preserves geometric detail
   through quantization for this dataset
"""
        else:
            diff_pct = ((avg_mm_mse - avg_us_mse) / avg_mm_mse) * 100
            conclusion += f"""
   Unit Sphere normalization resulted in lower reconstruction errors 
   (MSE ≈ {avg_us_mse:.6f}) compared to Min-Max normalization.
    
  Unit Sphere achieves {diff_pct:.1f}% lower MSE on average
  Uniform isotropic scaling better preserves geometric detail through
  quantization for this dataset
"""
        
        # Per-axis analysis
        avg_mm_mse_x = np.mean([r['mm_mse_x'] for r in all_results])
        avg_mm_mse_y = np.mean([r['mm_mse_y'] for r in all_results])
        avg_mm_mse_z = np.mean([r['mm_mse_z'] for r in all_results])
        
        axis_errors = [('X', avg_mm_mse_x), ('Y', avg_mm_mse_y), ('Z', avg_mm_mse_z)]
        max_axis = max(axis_errors, key=lambda x: x[1])
        
        conclusion += f"""
Per-Axis Error Analysis:
  • {max_axis[0]}-axis showed slightly higher error (MSE: {max_axis[1]:.8f})
  • This may be due to scale compression during normalization or
    quantization bin distribution along this axis

Quantization Quality:
  With {BINS} bins, both methods preserve geometric detail well
  Average reconstruction error is very low (< 0.001 for most meshes)
 

Recommendation:
  For this dataset, {'Min-Max' if avg_mm_mse < avg_us_mse else 'Unit Sphere'} normalization is ideal
  Error levels are acceptable for most 3D processing applications
  

{'='*70}
"""
        
        conclusion_path = os.path.join(OUTPUT_ROOT, "task3_conclusion.txt")
        with open(conclusion_path, 'w', encoding='utf-8') as f:
            f.write(conclusion)
        
        print(conclusion)
        print(f"Conclusion saved to: {conclusion_path}\n")
    
    print(f"{'='*70}")
    print(f"TASK 3 COMPLETE")
    print(f"{'='*70}\n")

In [ ]:
if __name__ == "__main__":
    process_all_meshes_task3()

In [ ]:
def generate_transformed_meshes(original_mesh, num_transformations=NUM_TRANSFORMATIONS):
    """
    Generate NUM_TRANSFORMATIONS-1 randomly transformed versions.
    Returns: list of meshes [original, transformed1, transformed2, ...]
    """
    print(f"  [1] Generating {num_transformations} mesh versions...")
    
    original_vertices = np.array(original_mesh.vertices)
    faces = np.array(original_mesh.faces)
    
    transformed_meshes = [original_mesh]  # Include original
    
    for i in range(num_transformations - 1):
        # Random rotation
        random_rotation = Rotation.random().as_matrix()
        
        # Random translation (±0.1 range as specified)
        random_translation = (np.random.rand(3) - 0.5) * 0.2
        
        # Apply transformation
        new_vertices = (original_vertices @ random_rotation.T) + random_translation
        
        # Create new mesh
        new_mesh = trimesh.Trimesh(new_vertices, faces, process=False)
        transformed_meshes.append(new_mesh)
    
    print(f"      ✓ Generated {num_transformations} versions (1 original + {num_transformations-1} transformed)")
    return transformed_meshes

In [ ]:
def compute_vertex_density(vertices, k=K_NEIGHBORS):
    """
    Compute local vertex density using KDTree.
    Returns: normalized density scores [0, 1] per vertex.
    """
    print(f"      Building KDTree...")
    tree = KDTree(vertices)
    
    print(f"      Querying {k}-nearest neighbors...")
    # Query k+1 because point itself is included
    distances, _ = tree.query(vertices, k=k+1)
    
    # Get average distance to k neighbors
    avg_distances = distances[:, 1:].mean(axis=1)  # Exclude self
    
    # Density = inverse of average distance
    safe_distances = np.maximum(avg_distances, 1e-9)
    density_scores = 1.0 / safe_distances
    
    # Normalize to [0, 1]
    min_score = density_scores.min()
    max_score = density_scores.max()
    range_score = max_score - min_score
    
    if range_score == 0:
        normalized_scores = np.ones_like(density_scores) * 0.5
    else:
        normalized_scores = (density_scores - min_score) / range_score
    
    print(f"      ✓ Density computed (range: [{normalized_scores.min():.3f}, {normalized_scores.max():.3f}])")
    return normalized_scores

In [ ]:
def adaptive_quantize(normalized_vertices, density_scores, base_bins=BINS, 
                     density_factor=ADAPTIVE_DENSITY_FACTOR):
    """
    Adaptive quantization: variable bins based on density.
    """
    # Shift to [0, 1] if needed
    norm_01 = normalized_vertices.copy()
    if norm_01.min() < -0.01:
        norm_01 = (norm_01 + 1.0) / 2.0
    
    norm_01 = np.clip(norm_01, 0.0, 1.0)
    
    # Compute local bin counts per vertex
    local_bins = base_bins * (1.0 + density_factor * density_scores)
    
    # Broadcast to (N, 3) for XYZ
    local_bins_3d = np.repeat(local_bins[:, np.newaxis], 3, axis=1)
    
    # Quantize with variable bins per vertex
    quantized = np.floor(norm_01 * (local_bins_3d - 1)).astype(np.int64)
    
    # Clip each vertex to its own max bin
    for i in range(len(quantized)):
        max_bin = int(local_bins_3d[i, 0] - 1)
        quantized[i] = np.clip(quantized[i], 0, max_bin)
    
    return quantized, local_bins_3d

def adaptive_dequantize(quantized_vertices, local_bins_3d):
    """
    Dequantize using variable bin counts.
    """
    # Dequantize to [0, 1]
    dequant_01 = quantized_vertices.astype(float) / (local_bins_3d - 1)
    
    # Shift back to [-1, 1]
    dequant_m1_1 = dequant_01 * 2.0 - 1.0
    
    return dequant_m1_1

In [ ]:
def plot_error_comparison(results_df, sample_name, sample_dir):
    """Plot Uniform vs Adaptive errors across transformations."""
    plot_path = os.path.join(sample_dir, "plots", "error_comparison.png")
    os.makedirs(os.path.dirname(plot_path), exist_ok=True)
    
    n_groups = len(results_df)
    index = np.arange(n_groups)
    bar_width = 0.35
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle(f'Task 4: Uniform vs Adaptive Quantization - {sample_name}',
                 fontsize=14, fontweight='bold')
    
    # MSE Plot
    ax1.bar(index - bar_width/2, results_df['uniform_mse'], bar_width,
            label='Uniform', color='royalblue', alpha=0.8)
    ax1.bar(index + bar_width/2, results_df['adaptive_mse'], bar_width,
            label='Adaptive', color='seagreen', alpha=0.8)
    ax1.set_xlabel('Transformation')
    ax1.set_ylabel('Mean Squared Error (MSE)')
    ax1.set_title('MSE Comparison')
    ax1.set_xticks(index)
    ax1.set_xticklabels([f"V{i}" for i in range(n_groups)])
    ax1.legend()
    ax1.grid(axis='y', alpha=0.3)
    ax1.set_yscale('log')
    
    # MAE Plot
    ax2.bar(index - bar_width/2, results_df['uniform_mae'], bar_width,
            label='Uniform', color='royalblue', alpha=0.8)
    ax2.bar(index + bar_width/2, results_df['adaptive_mae'], bar_width,
            label='Adaptive', color='seagreen', alpha=0.8)
    ax2.set_xlabel('Transformation')
    ax2.set_ylabel('Mean Absolute Error (MAE)')
    ax2.set_title('MAE Comparison')
    ax2.set_xticks(index)
    ax2.set_xticklabels([f"V{i}" for i in range(n_groups)])
    ax2.legend()
    ax2.grid(axis='y', alpha=0.3)
    ax2.set_yscale('log')
    
    plt.tight_layout()
    plt.savefig(plot_path, dpi=200, bbox_inches='tight')
    plt.close(fig)
    print(f"       Error comparison plot saved")

def plot_bin_distribution(density_scores, sample_name, sample_dir):
    """Plot distribution of adaptive bin sizes."""
    plot_path = os.path.join(sample_dir, "plots", "bin_distribution.png")
    
    local_bins = BINS * (1.0 + ADAPTIVE_DENSITY_FACTOR * density_scores)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    fig.suptitle(f'Task 4: Adaptive Bin Distribution - {sample_name}',
                 fontsize=14, fontweight='bold')
    
    # Histogram
    ax1.hist(local_bins, bins=50, color='seagreen', alpha=0.7, edgecolor='black')
    ax1.axvline(BINS, color='red', linestyle='--', label=f'Base bins ({BINS})')
    ax1.axvline(local_bins.mean(), color='blue', linestyle='--', label=f'Mean ({local_bins.mean():.0f})')
    ax1.set_xlabel('Number of Bins')
    ax1.set_ylabel('Frequency')
    ax1.set_title('Distribution of Bin Counts')
    ax1.legend()
    ax1.grid(axis='y', alpha=0.3)
    
    # Density vs Bins scatter
    ax2.scatter(density_scores, local_bins, s=6, c=density_scores, 
               cmap='viridis', alpha=0.5)
    ax2.set_xlabel('Normalized Density Score')
    ax2.set_ylabel('Number of Bins')
    ax2.set_title('Density vs Bin Count')
    ax2.grid(alpha=0.3)
    cbar = plt.colorbar(ax2.collections[0], ax=ax2)
    cbar.set_label('Density')
    
    plt.tight_layout()
    plt.savefig(plot_path, dpi=200, bbox_inches='tight')
    plt.close(fig)
    print(f"       Bin distribution plot saved")

def plot_normalization_invariance(invariance_data, sample_name, sample_dir):
    """Plot normalization consistency across transformations."""
    plot_path = os.path.join(sample_dir, "plots", "normalization_invariance.png")
    
    versions = list(range(len(invariance_data['centroids_mm'])))
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f'Task 4: Normalization Invariance - {sample_name}',
                 fontsize=14, fontweight='bold')
    
    # Min-Max Centroids
    centroids_mm = np.array(invariance_data['centroids_mm'])
    ax1.plot(versions, centroids_mm[:, 0], 'o-', label='X', color='red')
    ax1.plot(versions, centroids_mm[:, 1], 's-', label='Y', color='green')
    ax1.plot(versions, centroids_mm[:, 2], '^-', label='Z', color='blue')
    ax1.set_xlabel('Transformation Version')
    ax1.set_ylabel('Centroid Value')
    ax1.set_title('Min-Max: Centroid Variation')
    ax1.legend()
    ax1.grid(alpha=0.3)
    
    # Unit Sphere Centroids
    centroids_us = np.array(invariance_data['centroids_us'])
    ax2.plot(versions, centroids_us[:, 0], 'o-', label='X', color='red')
    ax2.plot(versions, centroids_us[:, 1], 's-', label='Y', color='green')
    ax2.plot(versions, centroids_us[:, 2], '^-', label='Z', color='blue')
    ax2.set_xlabel('Transformation Version')
    ax2.set_ylabel('Centroid Value')
    ax2.set_title('Unit Sphere: Centroid Variation')
    ax2.legend()
    ax2.grid(alpha=0.3)
    
    # Min-Max Scale/Range
    scales_mm = np.array(invariance_data['scales_mm'])
    ax3.plot(versions, scales_mm, 'o-', color='purple', linewidth=2)
    ax3.set_xlabel('Transformation Version')
    ax3.set_ylabel('Scale Value')
    ax3.set_title('Min-Max: Scale Variation')
    ax3.grid(alpha=0.3)
    
    # Unit Sphere Scale
    scales_us = np.array(invariance_data['scales_us'])
    ax4.plot(versions, scales_us, 'o-', color='orange', linewidth=2)
    ax4.set_xlabel('Transformation Version')
    ax4.set_ylabel('Scale Value')
    ax4.set_title('Unit Sphere: Scale Variation')
    ax4.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(plot_path, dpi=200, bbox_inches='tight')
    plt.close(fig)
    print(f"      ✓ Normalization invariance plot saved")

In [ ]:
def process_mesh_task4(mesh_path, sample_name):
    """
    Complete Task 4 pipeline for one mesh.
    
    Steps:
    1. Generate transformed versions
    2. Normalize each (both methods) and check invariance
    3. Compute density
    4. Quantize (uniform + adaptive)
    5. Reconstruct
    6. Compute errors
    7. Visualize
    """
    print(f"\n{'='*70}")
    print(f"Task 4 Processing: {sample_name}")
    print(f"{'='*70}")
    
    # Setup directories
    sample_dir = os.path.join(OUTPUT_ROOT_TASK4, sample_name)
    subdirs = ['transformations', 'normalized', 'quantized_uniform', 
               'quantized_adaptive', 'reconstructions', 'plots']
    for subdir in subdirs:
        os.makedirs(os.path.join(sample_dir, subdir), exist_ok=True)
    
    # Load original mesh
    original_mesh = trimesh.load(mesh_path, process=False)
    original_vertices = np.array(original_mesh.vertices)
    original_faces = np.array(original_mesh.faces)
    
    # STEP 1: Generate transformed meshes
    transformed_meshes = generate_transformed_meshes(original_mesh, NUM_TRANSFORMATIONS)
    
    # Save transformations
    for i, t_mesh in enumerate(transformed_meshes):
        t_mesh.export(os.path.join(sample_dir, "transformations", f"mesh_rot{i}.ply"))
    print(f"        Saved {NUM_TRANSFORMATIONS} transformed meshes")
    
    # Storage for results and invariance tracking
    results = []
    invariance_data = {
        'centroids_mm': [],
        'centroids_us': [],
        'scales_mm': [],
        'scales_us': []
    }
    
    # Process each transformation
    for version_id, t_mesh in enumerate(transformed_meshes):
        print(f"\n  --- Version {version_id} ---")
        t_vertices = np.array(t_mesh.vertices)
        
        # STEP 2: Normalize (both methods)
        print(f"    [2] Normalizing...")
        
        # Min-Max normalization
        norm_mm, vmin_mm, vmax_mm = minmax_normalize(t_vertices)
        scale_mm = np.max(vmax_mm - vmin_mm)  # Max range
        centroid_mm = (vmax_mm + vmin_mm) / 2
        
        # Unit Sphere normalization
        norm_us, centroid_us, scale_us = unit_sphere_normalize(t_vertices)
        
        # Track invariance metrics
        invariance_data['centroids_mm'].append(centroid_mm)
        invariance_data['centroids_us'].append(centroid_us)
        invariance_data['scales_mm'].append(scale_mm)
        invariance_data['scales_us'].append(scale_us)
        
        # Save normalized meshes
        trimesh.Trimesh(norm_mm, original_faces, process=False).export(
            os.path.join(sample_dir, "normalized", f"v{version_id}_minmax.ply"))
        trimesh.Trimesh(norm_us, original_faces, process=False).export(
            os.path.join(sample_dir, "normalized", f"v{version_id}_unitsphere.ply"))
        
        print(f"         Min-Max: centroid={centroid_mm}, scale={scale_mm:.4f}")
        print(f"         Unit Sphere: centroid={centroid_us}, scale={scale_us:.4f}")
        
        # STEP 3: Compute vertex density (on Unit Sphere normalized)
        print(f"    [3] Computing vertex density...")
        density_scores = compute_vertex_density(norm_us, k=K_NEIGHBORS)
        
        # Save density data
        np.save(os.path.join(sample_dir, "normalized", f"v{version_id}_density.npy"), 
                density_scores)
        
        # STEP 4: Quantization (Uniform baseline)
        print(f"    [4A] Uniform Quantization...")
        quant_uniform, _ = quantize(norm_us, bins=BINS)
        
        trimesh.Trimesh(quant_uniform.astype(float), original_faces, process=False).export(
            os.path.join(sample_dir, "quantized_uniform", f"v{version_id}.ply"))
        
        # STEP 4: Quantization (Adaptive)
        print(f"    [4B] Adaptive Quantization...")
        quant_adaptive, local_bins = adaptive_quantize(norm_us, density_scores, 
                                                   base_bins=BINS,
                                                   density_factor=ADAPTIVE_DENSITY_FACTOR)
        
        trimesh.Trimesh(quant_adaptive.astype(float), original_faces, process=False).export(
        os.path.join(sample_dir, "quantized_adaptive", f"v{version_id}.ply"))

        np.save(os.path.join(sample_dir, "quantized_adaptive", f"v{version_id}_local_bins.npy"), local_bins)
        
        print(f"         Adaptive bins range: [{local_bins.min():.0f}, {local_bins.max():.0f}]")
        
        # STEP 5: Reconstruction
        print(f"    [5] Reconstructing meshes...")
        
        # Uniform reconstruction
        dequant_uniform = dequantize(quant_uniform, bins=BINS, shifted=True)
        recon_uniform = denormalize_unitsphere(dequant_uniform, centroid_us, scale_us)
        
        # Adaptive reconstruction
        dequant_adaptive = adaptive_dequantize(quant_adaptive, local_bins)
        recon_adaptive = denormalize_unitsphere(dequant_adaptive, centroid_us, scale_us)
        
        # Save reconstructions
        trimesh.Trimesh(recon_uniform, original_faces, process=False).export(
            os.path.join(sample_dir, "reconstructions", f"v{version_id}_uniform.ply"))
        trimesh.Trimesh(recon_adaptive, original_faces, process=False).export(
            os.path.join(sample_dir, "reconstructions", f"v{version_id}_adaptive.ply"))
        
        # STEP 6: Compute errors (compare to original, not transformed)
        print(f"    [6] Computing reconstruction errors...")
        errors_uniform = compute_errors(t_vertices, recon_uniform)
        errors_adaptive = compute_errors(t_vertices, recon_adaptive)
        
        print(f"        Uniform  - MSE: {errors_uniform['mse']:.2e}, MAE: {errors_uniform['mae']:.2e}")
        print(f"        Adaptive - MSE: {errors_adaptive['mse']:.2e}, MAE: {errors_adaptive['mae']:.2e}")
        
        # Store results
        results.append({
            'version': version_id,
            'uniform_mse': errors_uniform['mse'],
            'uniform_mae': errors_uniform['mae'],
            'adaptive_mse': errors_adaptive['mse'],
            'adaptive_mae': errors_adaptive['mae'],
            'mse_x_uniform': errors_uniform['mse_axis'][0],
            'mse_y_uniform': errors_uniform['mse_axis'][1],
            'mse_z_uniform': errors_uniform['mse_axis'][2],
            'mse_x_adaptive': errors_adaptive['mse_axis'][0],
            'mse_y_adaptive': errors_adaptive['mse_axis'][1],
            'mse_z_adaptive': errors_adaptive['mse_axis'][2]
        })
    
    # STEP 7: Generate visualizations
    print(f"\n  [7] Generating visualizations...")
    results_df = pd.DataFrame(results)
    
    plot_error_comparison(results_df, sample_name, sample_dir)
    plot_bin_distribution(density_scores, sample_name, sample_dir)
    plot_normalization_invariance(invariance_data, sample_name, sample_dir)
    
    # Save invariance log as JSON
    invariance_log = {
        'centroids_mm': [c.tolist() for c in invariance_data['centroids_mm']],
        'centroids_us': [c.tolist() for c in invariance_data['centroids_us']],
        'scales_mm': invariance_data['scales_mm'],
        'scales_us': invariance_data['scales_us'],
        'centroid_mm_std': np.std(invariance_data['centroids_mm'], axis=0).tolist(),
        'centroid_us_std': np.std(invariance_data['centroids_us'], axis=0).tolist(),
        'scale_mm_std': float(np.std(invariance_data['scales_mm'])),
        'scale_us_std': float(np.std(invariance_data['scales_us']))
    }
    
    with open(os.path.join(sample_dir, "invariance_consistency_log.json"), 'w') as f:
        json.dump(invariance_log, f, indent=2)
    print(f"      Invariance consistency log saved")
    
    # Compute summary statistics
    summary = {
        'mesh': sample_name,
        'mean_uniform_mse': results_df['uniform_mse'].mean(),
        'std_uniform_mse': results_df['uniform_mse'].std(),
        'mean_adaptive_mse': results_df['adaptive_mse'].mean(),
        'std_adaptive_mse': results_df['adaptive_mse'].std(),
        'mean_uniform_mae': results_df['uniform_mae'].mean(),
        'mean_adaptive_mae': results_df['adaptive_mae'].mean(),
        'scale_us_variance': invariance_log['scale_us_std']
    }
    
    print(f"\n  [8] Summary Statistics:")
    print(f"      Uniform MSE:   {summary['mean_uniform_mse']:.2e} (±{summary['std_uniform_mse']:.2e})")
    print(f"      Adaptive MSE:  {summary['mean_adaptive_mse']:.2e} (±{summary['std_adaptive_mse']:.2e})")
    print(f"      Scale StdDev:  {summary['scale_us_variance']:.2e} (proves invariance)")
    
    print(f"\n{'='*70}")
    print(f"✓ Task 4 Completed: {sample_name}")
    print(f"{'='*70}\n")
    
    return summary, results_df

In [ ]:
def process_all_meshes_task4():
    """Process all meshes for Task 4."""
    mesh_files = [f for f in os.listdir(MESH_DIR) if f.endswith('.obj')]
    
    if not mesh_files:
        print(f"No .obj files found in {MESH_DIR}/")
        return
    
    print(f"\n{'='*70}")
    print(f"TASK 4: INVARIANCE & ADAPTIVE QUANTIZATION")
    print(f"{'='*70}")
    print(f"Found {len(mesh_files)} mesh file(s)")
    print(f"Configuration:")
    print(f"  - Transformations per mesh: {NUM_TRANSFORMATIONS}")
    print(f"  - Base bins: {BINS}")
    print(f"  - K-neighbors for density: {K_NEIGHBORS}")
    print(f"  - Adaptive density factor: {ADAPTIVE_DENSITY_FACTOR}")
    print(f"  - Output directory: {OUTPUT_ROOT_TASK4}/")
    
    all_summaries = []
    all_detailed_results = []
    
    for mesh_file in mesh_files:
        mesh_path = os.path.join(MESH_DIR, mesh_file)
        sample_name = os.path.splitext(mesh_file)[0]
        
        try:
            summary, detailed_df = process_mesh_task4(mesh_path, sample_name)
            all_summaries.append(summary)
            
            # Add mesh name to detailed results
            detailed_df['mesh'] = sample_name
            all_detailed_results.append(detailed_df)
            
        except Exception as e:
            print(f"\n✗ ERROR processing {mesh_file}: {e}")
            import traceback
            traceback.print_exc()
    
    # Aggregate and save results
    if all_summaries:
        print(f"\n{'='*70}")
        print(f"TASK 4: AGGREGATE RESULTS")
        print(f"{'='*70}")
        
        # Save summary CSV
        summary_df = pd.DataFrame(all_summaries)
        summary_path = os.path.join(OUTPUT_ROOT_TASK4, "task4_error_summary.csv")
        summary_df.to_csv(summary_path, index=False)
        print(f"\n  Summary saved: {summary_path}")
        
        # Save detailed results
        if all_detailed_results:
            detailed_df_all = pd.concat(all_detailed_results, ignore_index=True)
            detailed_path = os.path.join(OUTPUT_ROOT_TASK4, "task4_detailed_results.csv")
            detailed_df_all.to_csv(detailed_path, index=False)
            print(f"  Detailed results saved: {detailed_path}")
        
        # Calculate overall statistics
        avg_uniform_mse = summary_df['mean_uniform_mse'].mean()
        avg_adaptive_mse = summary_df['mean_adaptive_mse'].mean()
        avg_scale_variance = summary_df['scale_us_variance'].mean()
        avg_error_std = summary_df['std_uniform_mse'].mean()
        
        # Generate final conclusion
        conclusion = f"""
{'='*70}
TASK 4: FINAL ANALYSIS & CONCLUSION
{'='*70}

Dataset: {len(all_summaries)} meshes
Transformations per mesh: {NUM_TRANSFORMATIONS}
Total analyses: {len(all_summaries) * NUM_TRANSFORMATIONS}

{'='*70}
PART 1: TRANSFORMATION INVARIANCE
{'='*70}

Objective: Verify that normalization is robust to rotation and translation.

Method:
  • Applied {NUM_TRANSFORMATIONS} random transformations (rotation + translation)
  • Normalized each version using Unit Sphere method
  • Measured consistency of scale and centroid across transformations

Results:
  • Average Scale Standard Deviation: {avg_scale_variance:.6e}
  • Average MSE Standard Deviation: {avg_error_std:.6e}

**Conclusion - Part 1:**
The exceptionally low standard deviation ({avg_scale_variance:.6e}) confirms that
the Unit Sphere normalization is highly invariant to transformations. The
reconstruction error remains consistent regardless of the mesh's orientation
or position, validating the normalization's robustness.

{'='*70}
PART 2: ADAPTIVE QUANTIZATION EFFECTIVENESS
{'='*70}

Objective: Compare uniform vs. adaptive quantization strategies.

Configuration:
  • Uniform: All vertices use {BINS} bins
  • Adaptive: Bins range from {BINS} to {int(BINS * (1 + ADAPTIVE_DENSITY_FACTOR))}
  • Allocation: Based on local vertex density (KNN with k={K_NEIGHBORS})

Results:
  • Average Uniform MSE:   {avg_uniform_mse:.8f}
  • Average Adaptive MSE:  {avg_adaptive_mse:.8f}
"""
        
        if avg_adaptive_mse < avg_uniform_mse:
            improvement = ((avg_uniform_mse - avg_adaptive_mse) / avg_uniform_mse) * 100
            conclusion += f"""
**Conclusion - Part 2:**
✓ Adaptive quantization was **SUCCESSFUL**.

Adaptive quantization achieved {improvement:.2f}% lower MSE compared to uniform
quantization. By allocating more quantization bins to geometrically dense
regions, the adaptive method effectively:
  • Preserves fine details in complex areas
  • Reduces information loss during quantization
  • Improves overall reconstruction fidelity

This demonstrates that density-aware quantization is beneficial for meshes
with non-uniform geometric complexity.
"""
        else:
            difference = ((avg_adaptive_mse - avg_uniform_mse) / avg_uniform_mse) * 100
            conclusion += f"""
**Conclusion - Part 2:**
✗ Adaptive quantization did NOT outperform uniform quantization.

Adaptive quantization resulted in {abs(difference):.2f}% higher MSE compared
to uniform quantization. Possible explanations:
  • The meshes have relatively uniform geometric complexity
  • The base bin count ({BINS}) is already sufficient for this dataset
  • Density calculation (k={K_NEIGHBORS}) may need tuning
  • Adaptive overhead doesn't justify benefits for these meshes

For this dataset, uniform quantization with {BINS} bins appears optimal.
"""
        
        conclusion += f"""
{'='*70}
RECOMMENDATIONS
{'='*70}

1. Normalization Choice:
   ✓ Use Unit Sphere normalization for rotation/translation invariance
   → Essential for applications requiring consistent representation
      regardless of mesh orientation

2. Quantization Strategy:
"""
        
        if avg_adaptive_mse < avg_uniform_mse:
            conclusion += f"""   ✓ Use adaptive quantization for meshes with:
     - High geometric complexity variations
     - Detailed features in specific regions
     - Need for optimal detail preservation
   → Provides {improvement:.1f}% better reconstruction accuracy
"""
        else:
            conclusion += f"""   ✓ Use uniform quantization for:
     - Meshes with uniform complexity distribution
     - Applications prioritizing computational efficiency
     - Datasets where {BINS} bins are sufficient
   → Simpler, faster, and performs well on this dataset
"""
        
        conclusion += f"""
3. Parameter Tuning:
   • Base bins: {BINS} (current)
   • K-neighbors: {K_NEIGHBORS} (adjust based on mesh resolution)
   • Density factor: {ADAPTIVE_DENSITY_FACTOR} (increase for more adaptation)

{'='*70}
END OF ANALYSIS
{'='*70}
"""
        
        # Save conclusion
        conclusion_path = os.path.join(OUTPUT_ROOT_TASK4, "task4_conclusion.txt")
        with open(conclusion_path, 'w', encoding='utf-8') as f:
            f.write(conclusion)
        
        print(conclusion)
        print(f"\n  ✓ Analysis saved: {conclusion_path}")
    
    print(f"\n{'='*70}")
    print(f"TASK 4 COMPLETE")
    print(f"{'='*70}\n")


In [ ]:
if __name__ == "__main__":
    process_all_meshes_task4()  